--- Day 9: Rope Bridge ---

This rope bridge creaks as you walk along it. You aren't sure how old it is, or whether it can even support your weight.

It seems to support the Elves just fine, though. The bridge spans a gorge which was carved out by the massive river far below you.

You step carefully; as you do, the ropes stretch and twist. You decide to distract yourself by modeling rope physics; maybe you can even figure out where **not** to step.

Consider a rope with a knot at each end; these knots mark the **head** and the **tail** of the rope. If the head moves far enough away from the tail, the tail is pulled toward the head.

Due to nebulous reasoning involving [Planck lengths](https://en.wikipedia.org/wiki/Planck_units#Planck_length), you should be able to model the positions of the knots on a two-dimensional grid. Then, by following a hypothetical **series of motions** (your puzzle input) for the head, you can determine how the tail will move.

Due to the aforementioned Planck lengths, the rope must be quite short; in fact, the head (`H`) and tail (`T`) must **always be touching** (diagonally adjacent and even overlapping both count as touching):
```
....
.TH.
....

....
.H..
..T.
....

...
.H. (H covers T)
...
```
If the head is ever two steps directly up, down, left, or right from the tail, the tail must also move one step in that direction so it remains close enough:
```
.....    .....    .....
.TH.. -> .T.H. -> ..TH.
.....    .....    .....

...    ...    ...
.T.    .T.    ...
.H. -> ... -> .T.
...    .H.    .H.
...    ...    ...
```
Otherwise, if the head and tail aren't touching and aren't in the same row or column, the tail always moves one step diagonally to keep up:
```
.....    .....    .....
.....    ..H..    ..H..
..H.. -> ..... -> ..T..
.T...    .T...    .....
.....    .....    .....

.....    .....    .....
.....    .....    .....
..H.. -> ...H. -> ..TH.
.T...    .T...    .....
.....    .....    .....
```
You just need to work out where the tail goes as the head follows a series of motions. Assume the head and the tail both start at the same position, overlapping.

For example:
```
R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2
```
This series of motions moves the head **right** four steps, then **up** four steps, then **left** three steps, then **down** one step, and so on. After each step, you'll need to update the position of the tail if the step means the head is no longer adjacent to the tail. Visually, these motions occur as follows (s marks the starting position as a reference point):

== Initial State ==
```
......
......
......
......
H.....  (H covers T, s)

== R 4 ==

......
......
......
......
TH....  (T covers s)

......
......
......
......
sTH...

......
......
......
......
s.TH..

......
......
......
......
s..TH.

== U 4 ==

......
......
......
....H.
s..T..

......
......
....H.
....T.
s.....

......
....H.
....T.
......
s.....

....H.
....T.
......
......
s.....

== L 3 ==

...H..
....T.
......
......
s.....

..HT..
......
......
......
s.....

.HT...
......
......
......
s.....

== D 1 ==

..T...
.H....
......
......
s.....

== R 4 ==

..T...
..H...
......
......
s.....

..T...
...H..
......
......
s.....

......
...TH.
......
......
s.....

......
....TH
......
......
s.....

== D 1 ==

......
....T.
.....H
......
s.....

== L 5 ==

......
....T.
....H.
......
s.....

......
....T.
...H..
......
s.....

......
......
..HT..
......
s.....

......
......
.HT...
......
s.....

......
......
HT....
......
s.....

== R 2 ==

......
......
.H....  (H covers T)
......
s.....

......
......
.TH...
......
s.....
```
After simulating the rope, you can count up all of the positions the **tail visited at least once**. In this diagram, `s` again marks the starting position (which the tail also visited) and `#` marks other positions the tail visited:
```
..##..
...##.
.####.
....#.
s###..
```
So, there are `13` positions the tail visited at least once.

Simulate your complete hypothetical series of motions. **How many positions does the tail of the rope visit at least once?**

In [1]:
# Data Load
example_lines = [
    "R 4",
    "U 4",
    "L 3",
    "D 1",
    "R 4",
    "D 1",
    "L 5",
    "R 2",
]

with open("day_9_input.txt") as _file:
    data_lines = _file.readlines()

In [2]:
# Common Code
from __future__ import annotations
import typing
from collections import defaultdict


class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __members(self):
        return (self.x, self.y)

    def __repr__(self):
        return str(self.__members())

    def dist_x(self, other):
        return self.x - other.x

    def dist_y(self, other):
        return self.y - other.y

    def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return Vector(self.x - other.x, self.y - other.y)

    def __eq__(self, other):
        return self.__members() == other.__members()

    def __hash__(self):
        return hash(self.__members())

    def dist_manhattan(self, other: Vector):
        dist = self - other
        return abs(dist.x) + abs(dist.y)

    @classmethod
    def zero(cls):
        return cls(0, 0)

    @classmethod
    def one(cls):
        return cls(1, 1)


class Marker:
    mark: Vector

    def __init__(self):
        self.counts = defaultdict(lambda: 0)
        self._mark = None
        self.mark = Vector.zero()
        # self._mark = Vector.zero()
        # self.counts[self._mark] += 1

    @property
    def mark(self):
        return self._mark

    @mark.setter
    def mark(self, value):
        self._mark = value
        self.counts[value] += 1


class Direction(Vector):
    _DIRECTIONS = {
        "R": (1, 0),
        "L": (-1, 0),
        "U": (0, -1),
        "D": (0, 1)
    }
    @classmethod
    def parse(cls, symbol):
        if symbol in cls._DIRECTIONS:
            return cls(*cls._DIRECTIONS[symbol])
        raise RuntimeError(f"Unexpected symbol: {symbol}")


class Move:
    def __init__(self, direction: Direction, quantity):
        self.direction = direction
        self.quantity = quantity

    @classmethod
    def parse(cls, line):
        symbol, quantity = line.split()
        return cls(Direction.parse(symbol), int(quantity))


class Rope:
    _locations: dict[Vector, int]

    def __init__(self, moves: typing.Iterable[Move]):
        self._moves = moves

        self._head = Marker()
        self._tail = Marker()

        self._top_left = Vector.zero()
        self._bot_right = Vector.zero()

    @classmethod
    def parse(cls, lines):
        return cls(Move.parse(line.strip()) for line in lines)

    # def _mark_positions(self):
    #     self._head_locations[self._head] += 1
    #     self._tail_locations[self._tail] += 1

    def _apply_single_move(self, move: Direction):
        new_position: Vector = self._head.mark + move

        self._top_left.x = min(new_position.x, self._top_left.x)
        self._top_left.y = min(new_position.y, self._top_left.y)
        self._bot_right.x = max(new_position.x, self._bot_right.x)
        self._bot_right.y = max(new_position.y, self._bot_right.y)

        # if new_position.dist_manhattan(self._tail.mark) > 1:
        tail_diff = new_position - self._tail.mark
        tail_diff.x = abs(tail_diff.x)
        tail_diff.y = abs(tail_diff.y)

        if tail_diff.x > 1 or tail_diff.y > 1:
            self._tail.mark = self._head.mark

        self._head.mark = new_position

    def _apply_move(self, move: Move, display: bool):
        if display:
            print(f"== {move.direction} {move.quantity} ==")
            print()
        for _ in range(move.quantity):
            self._apply_single_move(move.direction)
            if display:
                self._print_state()

    def _print_state(self, show_tail_positions=False):
        dimensions = self._bot_right - self._top_left + Vector.one()
        board = ["." * dimensions.x for _ in range(dimensions.y)]

        zero_index = Vector.zero() - self._top_left

        def replace_char_at(pos: Vector, new_chr):
            pos += zero_index
            string = board[pos.y]
            board[pos.y] = string[:pos.x] + new_chr + string[pos.x + 1:]

        replace_char_at(Vector.zero(), "s")
        replace_char_at(self._tail.mark, "T")
        replace_char_at(self._head.mark, "H")

        if show_tail_positions:
            for position in self._tail.counts.keys():
                replace_char_at(position, "#")


        for row in board:
            print(row)
        print()

    def apply_moves(self, display=False):
        if display:
            print("== INITIAL STATE ==")
            print()
            self._print_state()

        for move in self._moves:
            self._apply_move(move, display)

    def get_tail_location_count(self):
        return len(self._tail.counts)

    def print_tail_locations(self):
        self._print_state(show_tail_positions=True)

In [3]:
# Solution 1 Code
def solution_1(lines, display):
    rope = Rope.parse(lines)

    rope.apply_moves(display)

    if display:
        rope.print_tail_locations()

    return rope.get_tail_location_count()


def test_solution_1():
    actual = solution_1(example_lines, True)
    expected = 13

    print(actual, expected)
    assert actual == expected
test_solution_1()

== INITIAL STATE ==

H

== (1, 0) 4 ==

TH

sTH

s.TH

s..TH

== (0, -1) 4 ==

....H
s..T.

....H
....T
s....

....H
....T
.....
s....

....H
....T
.....
.....
s....

== (-1, 0) 3 ==

...H.
....T
.....
.....
s....

..HT.
.....
.....
.....
s....

.HT..
.....
.....
.....
s....

== (0, 1) 1 ==

..T..
.H...
.....
.....
s....

== (1, 0) 4 ==

..T..
..H..
.....
.....
s....

..T..
...H.
.....
.....
s....

.....
...TH
.....
.....
s....

......
....TH
......
......
s.....

== (0, 1) 1 ==

......
....T.
.....H
......
s.....

== (-1, 0) 5 ==

......
....T.
....H.
......
s.....

......
....T.
...H..
......
s.....

......
......
..HT..
......
s.....

......
......
.HT...
......
s.....

......
......
HT....
......
s.....

== (1, 0) 2 ==

......
......
.H....
......
s.....

......
......
.TH...
......
s.....

..##..
...##.
.####.
....#.
####..

13 13


In [4]:
# Solution 1
solution_1(data_lines, False)

6090

_Problem 2_

In [5]:
# Solution 2 Code
def solution_2(lines):
    return ""

def test_solution_2():
    actual = solution_2(example_lines)
    expected = ""
    
    print(actual, expected)
    assert actual == expected
test_solution_2()

In [6]:
# Solution 2
solution_2(data_lines)

''